# Clean Time Logs

In [ ]:
import sys
sys.path.append("../../scripts")

from pathlib import Path

import pandas as pd
import numpy as np
import pingouin as pg

import biopsykit as bp
from biopsykit.utils.file_handling import get_subject_dirs
from biopsykit.utils.time import timedelta_to_time, get_time_from_date

from time_logs import process_time_log

from tqdm.notebook import tqdm

%load_ext autoreload
%autoreload 2
%matplotlib widget

In [ ]:
base_path = Path("../../../../../../HealthPsychology_D03/Data/2021_06_Micro_Prestudy")
data_path = base_path.joinpath("data_per_subject")
subject_dirs = get_subject_dirs(data_path, "VP_*")

subject_dirs

In [ ]:
for subject_dir in tqdm(subject_dirs):
    subject_id = subject_dir.name
    for condition in ["control", "stress"]:
        timelog_dir = subject_dir.joinpath("{}/time_log".format(condition))
        cleaned_dir = timelog_dir.joinpath("cleaned")
        
        timelogs = list(cleaned_dir.glob("*.csv"))
        if len(timelogs) == 0:
            print(f"No time logs for {subject_id}-{condition}.")
            continue
        timelog = process_time_log(list(cleaned_dir.glob("*.csv"))[0])
        timelog.columns = ["_".join(col) for col in timelog.columns]
        
        timelog = timelog.apply(lambda series: timedelta_to_time(get_time_from_date(series)))

        processed_dir = timelog_dir.joinpath("processed")
        processed_dir.mkdir(exist_ok=True)
        timelog.to_csv(processed_dir.joinpath("time_log_{}.csv".format(subject_id)))